In [10]:
!pip install --upgrade "google-cloud-bigquery>=3.31.0" "google-cloud-bigquery-storage>=2.30.0,<3.0.0"

  Using cached protobuf-6.32.0-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
Using cached protobuf-6.32.0-cp39-abi3-manylinux2014_x86_64.whl (322 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.25.1 which is incompatible.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 6.32.0 which is incompatible.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.32.0 which is incompatibl

In [11]:
!pip install protobuf==4.25.3 -q --force-reinstall

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.74.0 requires protobuf<7.0.0,>=6.31.1, but you have protobuf 4.25.3 which is incompatible.
google-cloud-automl 1.0.1 requires google-api-core[grpc]<2.0.0dev,>=1.14.0, but you have google-api-core 2.25.1 which is incompatible.


In [12]:
!pip install "rich<14,>=12.4.4"

In [13]:
!pip install -qU transformers datasets accelerate peft bitsandbytes trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.7 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
import torch

from trl import SFTTrainer, SFTConfig
from datasets import load_dataset, Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoModelForCausalLM,
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training

2025-08-19 10:33:41.560441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755599621.825867      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755599621.903298      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import pandas as pd
import json
import os
from pprint import pprint

In [16]:
# 데이터셋 설정
with open("/kaggle/input/json-data/interview_json_data.json", "r", encoding="utf-8") as f:
    interview_json_data = json.load(f)

pprint(interview_json_data[0])

{'answer': '네, 명확한 3년 성장 계획을 가지고 있습니다. 크게 세 가지 목표입니다. 첫째, 현재 제가 주력으로 다루는 백엔드 '
           '시스템 아키텍처 역량을 고도화하여, 대규모 트래픽 분산 처리 및 고가용성 설계 전문가가 되는 것입니다. 특히, '
           'MSA(Microservices Architecture) 환경에서 서비스 간 트랜잭션 일관성 유지 및 분산 추적 '
           '시스템(Distributed Tracing) 구축 경험을 심화하고 싶습니다. 이를 위해 특정 클라우드 플랫폼(예: '
           'AWS Solution Architect Professional) 인증 취득 및 관련 오픈소스 프로젝트에 기여하는 것을 '
           '목표로 하고 있습니다. 둘째, 데이터 기반 의사결정 역량을 강화하여, 개발하는 서비스의 사용자 행동 패턴 분석 및 '
           '비즈니스 성과 지표 도출에 직접적으로 기여하고자 합니다. 이를 위해 데이터 파이프라인 구축 및 시각화 도구 활용 능력을 '
           '향상시키고, AB 테스트 설계 및 분석 리더 역할을 수행하고 싶습니다. 셋째, 기술 리더십 역량을 키워 주니어 개발자 '
           '멘토링 및 팀 내 기술 스택 도입을 주도하는 역할을 하고 싶습니다. 단순히 코드를 잘 짜는 것을 넘어, 팀원들의 성장을 '
           '돕고 기술 부채를 줄이며, 장기적인 아키텍처 방향성을 제시하는 시니어 엔지니어로 자리매김하고 싶습니다. 이 목표들을 '
           '달성하기 위해 지속적으로 학습하고, 회사에 실질적인 가치를 제공할 수 있도록 노력할 것입니다.',
 'feedback': "3년 성장 계획을 '세 가지 목표'로 명확히 구조화하여 제시한 점이 매우 인상 깊습니다. 각 목표가 백엔드 아키텍처 "
             '고도화, 데이터 기반 의사결정 역량 강화, 기술 리더십 역량 강화 등 시니어 엔지니어로

In [17]:
# 데이터셋 llama3 에 맞게 설정
def create_format_dataset(json_data_list):
    formatted_data = []

    system_prompt = """당신은 전문 면접 평가자입니다. 당신의 임무는 제공된 면접 질문과 지원자의 답변을 분석하는 것입니다.
이 분석을 바탕으로 'score', 'feedback', 'improve'이라는 세 가지 키를 포함하는 JSON 객체를 생성해야 합니다.
- 'score': 답을 평가하는 숫자 점수 (1.0 ~ 5.0).
- 'feedback': 답변의 강점에 대한 상세하고 건설적인 피드백.
- 'improve': 답변에서 개선할 수 있는 사항에 대한 제안."""
    
    for item in json_data_list:
        # 'user' 부분: 질문과 답변
        user_content = f"""
        ### 질문:
        {item['question']}
        ### 답변:
        {item['answer']}"""

        # 'assistant' 부분: 평가 JSON
        assistant_content_dict = {
            "score": item['score'],
            "feedback": item['feedback'],
            "improve": item['improve']
        }
        assistant_content = json.dumps(assistant_content_dict, ensure_ascii=False, indent=2)

        # Llama 3 템플릿에 맞게 messages 리스트 구성
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content}
        ]

        formatted_data.append({"messages": messages})

    return formatted_data


In [18]:
# 데이터셋
training_data = create_format_dataset(interview_json_data)
# Hugging Face Dataset 객체로 변환
dataset = Dataset.from_pandas(pd.DataFrame(training_data))

# 변환된 데이터 확인
print(dataset[0]['messages'])

train_test_split = dataset.train_test_split(test_size=0.1)

# DatasetDict 생성
split_dataset = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})



[{'content': "당신은 전문 면접 평가자입니다. 당신의 임무는 제공된 면접 질문과 지원자의 답변을 분석하는 것입니다.\n이 분석을 바탕으로 'score', 'feedback', 'improve'이라는 세 가지 키를 포함하는 JSON 객체를 생성해야 합니다.\n- 'score': 답을 평가하는 숫자 점수.\n- 'feedback': 답변의 강점에 대한 상세하고 건설적인 피드백.\n- 'improve': 답변에서 개선할 수 있는 사항에 대한 제안.", 'role': 'system'}, {'content': '\n        ### 질문:\n        지원자님께서 지금부터 삼 년 뒤까지 즉 삼 년 이내로 주요 성장 계획이나 목표가 있으신가요 이러한 목표가 있다면 무엇인지 말씀해 주시겠습니까\n        ### 답변:\n        네, 명확한 3년 성장 계획을 가지고 있습니다. 크게 세 가지 목표입니다. 첫째, 현재 제가 주력으로 다루는 백엔드 시스템 아키텍처 역량을 고도화하여, 대규모 트래픽 분산 처리 및 고가용성 설계 전문가가 되는 것입니다. 특히, MSA(Microservices Architecture) 환경에서 서비스 간 트랜잭션 일관성 유지 및 분산 추적 시스템(Distributed Tracing) 구축 경험을 심화하고 싶습니다. 이를 위해 특정 클라우드 플랫폼(예: AWS Solution Architect Professional) 인증 취득 및 관련 오픈소스 프로젝트에 기여하는 것을 목표로 하고 있습니다. 둘째, 데이터 기반 의사결정 역량을 강화하여, 개발하는 서비스의 사용자 행동 패턴 분석 및 비즈니스 성과 지표 도출에 직접적으로 기여하고자 합니다. 이를 위해 데이터 파이프라인 구축 및 시각화 도구 활용 능력을 향상시키고, AB 테스트 설계 및 분석 리더 역할을 수행하고 싶습니다. 셋째, 기술 리더십 역량을 키워 주니어 개발자 멘토링 및 팀 내 기술 스택 도입을 주도하는 역할을 하고 싶습니다. 단순히 코드를 잘 짜는 것을 넘어, 팀원

In [ ]:
# HuggingFace 로그인
from huggingface_hub import login
login(token=os.getenv("HF_TOKEN"))

In [20]:
# 1. 모델 및 토크나이저 설정
model_name = "meta-llama/Llama-3.1-8B-Instruct"

In [21]:
# 2. QLoRA 설정 (VRAM 최소화)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [22]:
# 3. 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [23]:
# 학습 시작 전, 검증용 코드
# dataset은 Dataset 객체
sample_messages = dataset[0]['messages']

formatted_text = tokenizer.apply_chat_template(
    sample_messages,
    tokenize=False,
    add_generation_prompt=False
)

print("--- 템플릿 적용 결과 ---")
print(formatted_text)
print("----------------------")

--- 템플릿 적용 결과 ---
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

당신은 전문 면접 평가자입니다. 당신의 임무는 제공된 면접 질문과 지원자의 답변을 분석하는 것입니다.
이 분석을 바탕으로 'score', 'feedback', 'improve'이라는 세 가지 키를 포함하는 JSON 객체를 생성해야 합니다.
- 'score': 답을 평가하는 숫자 점수.
- 'feedback': 답변의 강점에 대한 상세하고 건설적인 피드백.
- 'improve': 답변에서 개선할 수 있는 사항에 대한 제안.<|eot_id|><|start_header_id|>user<|end_header_id|>

### 질문:
        지원자님께서 지금부터 삼 년 뒤까지 즉 삼 년 이내로 주요 성장 계획이나 목표가 있으신가요 이러한 목표가 있다면 무엇인지 말씀해 주시겠습니까
        ### 답변:
        네, 명확한 3년 성장 계획을 가지고 있습니다. 크게 세 가지 목표입니다. 첫째, 현재 제가 주력으로 다루는 백엔드 시스템 아키텍처 역량을 고도화하여, 대규모 트래픽 분산 처리 및 고가용성 설계 전문가가 되는 것입니다. 특히, MSA(Microservices Architecture) 환경에서 서비스 간 트랜잭션 일관성 유지 및 분산 추적 시스템(Distributed Tracing) 구축 경험을 심화하고 싶습니다. 이를 위해 특정 클라우드 플랫폼(예: AWS Solution Architect Professional) 인증 취득 및 관련 오픈소스 프로젝트에 기여하는 것을 목표로 하고 있습니다. 둘째, 데이터 기반 의사결정 역량을 강화하여, 개발하는 서비스의 사용자 행동 패턴 분석 및 비즈니스 성과 지표 도출에 직접적으로 기여하고자 합니다. 이를 위해 데이터 파이프라인 구축 및 시각화 도구 활용 능력

In [34]:
# 5. Trainer 설정
output_dir = "/kaggle/working/llama_interview_model"
training_args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    # learning_rate=5e-6,# 학습률
    learning_rate=2e-5,           # <-- 더 낮은 학습률
    max_grad_norm=1.0,            # <-- 경사 클리핑 추가
    # fp16=True,
    logging_steps=5,
    optim="paged_adamw_8bit",  # QLora 양자화 설정 (VRAM 부족한 경우)
    # optim="adamw_torch",
    dataset_text_field="messages", # 데이터셋에서 'messages' 필드를 사용하도록 지정
    packing=False,
    max_length=1024,           # 시퀀스 최대 길이

    # --- 스케줄러 워밍업 ---
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,

    # --- 체크포인트 저장 설정 (스텝 기반) ---
    save_strategy="steps",
    save_steps=20,          # 20 스텝마다 저장
    save_total_limit=5,     # 최대 5개의 체크포인트만 유지
)

In [25]:
# 4. LoRA 설정
lora_config = LoraConfig(
    r=16, lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)
model = prepare_model_for_kbit_training(model)
# peft_model = get_peft_model(model, lora_config)
# peft_model.print_trainable_parameters()

In [26]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset['train'], # <-- 훈련 데이터셋
    eval_dataset=split_dataset['test'],   # <-- 평가 데이터셋
    processing_class=tokenizer,
    peft_config=lora_config,    # LoRA 설정을 여기에 전달
)

Tokenizing train dataset:   0%|          | 0/1157 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1157 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/129 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/129 [00:00<?, ? examples/s]

In [27]:
import wandb

In [28]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("WANDB_API_KEY")

In [29]:
wandb.login(key=secret_value_0)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sogno6037 (sogno6037-university-of-ulsan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [33]:
# 학습 시작 ( resume_from_checkpoint=True )
trainer.train()

# 학습된 모델 저장
trainer.save_model("/kaggle/working/final-llama3-model")

ValueError: No valid checkpoint found in output directory (/kaggle/working/llama_interview_model)

In [31]:
# !zip -r output.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/llama_interview_model/ (stored 0%)
  adding: kaggle/working/llama_interview_model/runs/ (stored 0%)
  adding: kaggle/working/llama_interview_model/runs/Aug19_10-37-04_143a3fbda84c/ (stored 0%)
  adding: kaggle/working/llama_interview_model/runs/Aug19_10-37-04_143a3fbda84c/events.out.tfevents.1755599835.143a3fbda84c.36.0 (deflated 61%)
  adding: kaggle/working/wandb/ (stored 0%)
  adding: kaggle/working/wandb/run-20250819_103715-rpn09rgv/ (stored 0%)
  adding: kaggle/working/wandb/run-20250819_103715-rpn09rgv/run-rpn09rgv.wandb (deflated 89%)
  adding: kaggle/working/wandb/run-20250819_103715-rpn09rgv/tmp/ (stored 0%)
  adding: kaggle/working/wandb/run-20250819_103715-rpn09rgv/tmp/code/ (stored 0%)
  adding: kaggle/working/wandb/run-20250819_103715-rpn09rgv/logs/ (stored 0%)
  adding: kaggle/working/wandb/run-20250819_103715-rpn09rgv/logs/debug.log (deflated 66%)
  adding: kaggle/working/wandb/run-20250819_103715-rpn09rgv/lo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
